In [ ]:
import pandas as pd
import json
with open('large_model_results.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]
df = pd.DataFrame(data)
df.head()

,query,answers,generation,passages,metadata,id
0,question: What is a feed forward network answe...,[A neural network architecture where the compu...,A feed forward network is a multilayer network...,"[{'id': '3226', 'contributed_by': 'group 2', '...",{},1
1,question: What is the main advantage of neural...,[Neural networks are typically more powerful c...,"Firstly, neural network is a supervised machin...","[{'id': '2340', 'contributed_by': 'group 1', '...",{},2
2,question: What is the property of a Sigmoid fu...,[The Sigmoid function maps the input into the ...,Sigmoid functions are exponential functions. T...,"[{'id': '2374', 'contributed_by': 'group 1', '...",{},3
3,question: Describe the vanishing gradient prob...,[Since we train networks by propagating an err...,The vanishing gradient problem is a problem wi...,"[{'id': '3287', 'contributed_by': 'group 2', '...",{},4
4,question: What is a fully connected layer in n...,[It is a layer such that each unit in the laye...,"A fully connected layer is fully connected, me...","[{'id': '3152', 'contributed_by': 'group 2', '...",{},5


In [ ]:
df['answers'] = df['answers'].str.join('')

In [ ]:
df.head()

,query,answers,generation,passages,metadata,id
0,question: What is a feed forward network answe...,A neural network architecture where the comput...,A feed forward network is a multilayer network...,"[{'id': '3226', 'contributed_by': 'group 2', '...",{},1
1,question: What is the main advantage of neural...,Neural networks are typically more powerful cl...,"Firstly, neural network is a supervised machin...","[{'id': '2340', 'contributed_by': 'group 1', '...",{},2
2,question: What is the property of a Sigmoid fu...,The Sigmoid function maps the input into the r...,Sigmoid functions are exponential functions. T...,"[{'id': '2374', 'contributed_by': 'group 1', '...",{},3
3,question: Describe the vanishing gradient prob...,Since we train networks by propagating an erro...,The vanishing gradient problem is a problem wi...,"[{'id': '3287', 'contributed_by': 'group 2', '...",{},4
4,question: What is a fully connected layer in n...,It is a layer such that each unit in the layer...,"A fully connected layer is fully connected, me...","[{'id': '3152', 'contributed_by': 'group 2', '...",{},5


In [ ]:
df.shape

(100, 6)

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from sentence_transformers import CrossEncoder

# Load the pre-trained model
model = CrossEncoder('cross-encoder/stsb-roberta-base')

sentence_pairs = []
for sentence1, sentence2 in zip(df['answers'], df['generation']):
    sentence_pairs.append([sentence1, sentence2])
    
df['SBERT CrossEncoder_score'] = model.predict(sentence_pairs, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df.head()

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score
0,question: What is a feed forward network answe...,A neural network architecture where the comput...,A feed forward network is a multilayer network...,"[{'id': '3226', 'contributed_by': 'group 2', '...",{},1,0.505133
1,question: What is the main advantage of neural...,Neural networks are typically more powerful cl...,"Firstly, neural network is a supervised machin...","[{'id': '2340', 'contributed_by': 'group 1', '...",{},2,0.598798
2,question: What is the property of a Sigmoid fu...,The Sigmoid function maps the input into the r...,Sigmoid functions are exponential functions. T...,"[{'id': '2374', 'contributed_by': 'group 1', '...",{},3,0.765750
3,question: Describe the vanishing gradient prob...,Since we train networks by propagating an erro...,The vanishing gradient problem is a problem wi...,"[{'id': '3287', 'contributed_by': 'group 2', '...",{},4,0.495402
4,question: What is a fully connected layer in n...,It is a layer such that each unit in the layer...,"A fully connected layer is fully connected, me...","[{'id': '3152', 'contributed_by': 'group 2', '...",{},5,0.725582


In [ ]:
average_score=df['SBERT CrossEncoder_score'].mean()

In [ ]:
average_score

0.5884032

In [ ]:
max_score=df['SBERT CrossEncoder_score'].max()

In [ ]:
max_score

0.955481

In [ ]:
df[df['SBERT CrossEncoder_score']==max_score]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score
44,question: What is an RNN language model answer...,RNN language models process the input sequence...,An RNN language model is a language model that...,"[{'id': '3290', 'contributed_by': 'group 2', '...",{},45,0.955481


In [ ]:
min_score=df['SBERT CrossEncoder_score'].min()

In [ ]:
min_score

0.20883437

In [ ]:
df[df['SBERT CrossEncoder_score']==min_score]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score
94,question: What are examples of pairwise sequen...,"Recognizing textual entailment, or whether one...","Biological sequence classification, spam detec...","[{'id': '3183', 'contributed_by': 'group 2', '...",{},95,0.208834


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def calculate_f1_score(row):
    generated_answer = row['generation']
    ground_truth_answer = row['answers']
    f1 = f1_score([ground_truth_answer], [generated_answer], average='macro')

    return f1

# Apply the calculate_f1_score function to each row of the DataFrame to create a new column called "f1_score"
df['f1_score'] = df.apply(calculate_f1_score, axis=1)

In [ ]:
min_score

0.20883437

In [ ]:
import regex
import string
from collections import Counter

In [ ]:
def normalize_answer(s: str) -> str:
    def remove_articles(text):
        return regex.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def f1(prediction, ground_truth, normalize_fn):
    prediction_tokens = normalize_fn(prediction).split()
    ground_truth_tokens = normalize_fn(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


In [ ]:
df['f1_score'] = df.apply(lambda x: f1(x['generation'], x['answers'], normalize_answer), axis=1)

In [ ]:
df

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
0,question: What is a feed forward network answe...,A neural network architecture where the comput...,A feed forward network is a multilayer network...,"[{'id': '3226', 'contributed_by': 'group 2', '...",{},1,0.505133,0.210526
1,question: What is the main advantage of neural...,Neural networks are typically more powerful cl...,"Firstly, neural network is a supervised machin...","[{'id': '2340', 'contributed_by': 'group 1', '...",{},2,0.598798,0.146341
2,question: What is the property of a Sigmoid fu...,The Sigmoid function maps the input into the r...,Sigmoid functions are exponential functions. T...,"[{'id': '2374', 'contributed_by': 'group 1', '...",{},3,0.765750,0.424242
3,question: Describe the vanishing gradient prob...,Since we train networks by propagating an erro...,The vanishing gradient problem is a problem wi...,"[{'id': '3287', 'contributed_by': 'group 2', '...",{},4,0.495402,0.142857
4,question: What is a fully connected layer in n...,It is a layer such that each unit in the layer...,"A fully connected layer is fully connected, me...","[{'id': '3152', 'contributed_by': 'group 2', '...",{},5,0.725582,0.607143
...,...,...,...,...,...,...,...,...
95,question: What is an example of span-based app...,"Identification of spans of interest, entity re...",Span-based language model training. In this ex...,"[{'id': '3567', 'contributed_by': 'group 2', '...",{},96,0.525455,0.195122
96,question: What are some of the potential harms...,"Language models can generate toxic language, s...",Language models can generate toxic language. G...,"[{'id': '3583', 'contributed_by': 'group 2', '...",{},97,0.605022,0.291667
97,question: What cause biases in language models...,"For example, the training data may contain a b...","Several factors, including the number of words...","[{'id': '1720', 'contributed_by': 'group 1', '...",{},98,0.564706,0.058824
98,question: What are ways to mitigate the biases...,To carefully analyze the data used to pre-trai...,Several methods are available to mitigate the ...,"[{'id': '1833', 'contributed_by': 'group 1', '...",{},99,0.342975,0.139535


In [ ]:
max_score_f1=df['f1_score'].max()

In [ ]:
min_score_f1=df['f1_score'].min()

In [ ]:
avg_score_f1=df['f1_score'].mean()

In [ ]:
print('*'*45 + 'F1 Score'+ '*'*45)
print(f'max score | {max_score_f1} | min score | {min_score_f1} | avg_score | {avg_score_f1}')
print('\n')
print('*'*45 + 'Semantic Similarity Score'+ '*'*45)
print(f'max score | {max_score} | min score | {min_score} | avg_score | {average_score}')

*********************************************F1 Score*********************************************
max score | 0.8947368421052632 | min score | 0.0 | avg_score | 0.3136268515195137


*********************************************Semantic Similarity Score*********************************************
max score | 0.9554809927940369 | min score | 0.20883436501026154 | avg_score | 0.5884032249450684


In [ ]:
df[df['f1_score'] == max_score_f1]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
16,question: What is the perceptron answer: <extr...,The perceptron is a very simple neural unit th...,0 or 1? The perceptron is a very simple neural...,"[{'id': '2929', 'contributed_by': 'group 2', '...",{},17,0.7642,0.894737


In [ ]:
df[df['SBERT CrossEncoder_score'] == max_score]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
44,question: What is an RNN language model answer...,RNN language models process the input sequence...,An RNN language model is a language model that...,"[{'id': '3290', 'contributed_by': 'group 2', '...",{},45,0.955481,0.833333


In [ ]:
df[df['f1_score'] == min_score_f1]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
94,question: What are examples of pairwise sequen...,"Recognizing textual entailment, or whether one...","Biological sequence classification, spam detec...","[{'id': '3183', 'contributed_by': 'group 2', '...",{},95,0.208834,0.0
99,question: What are some examples of bidirectio...,"BERT, RoBERTa, and SpanBERT",Several examples of bidirectional transformer ...,"[{'id': '3505', 'contributed_by': 'group 2', '...",{},100,0.779548,0.0


In [ ]:
df[df['SBERT CrossEncoder_score'] == min_score]

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
94,question: What are examples of pairwise sequen...,"Recognizing textual entailment, or whether one...","Biological sequence classification, spam detec...","[{'id': '3183', 'contributed_by': 'group 2', '...",{},95,0.208834,0.0


In [ ]:
df_sorted = df.sort_values('f1_score', ascending=True)

In [ ]:
df_sorted.head(50)

,query,answers,generation,passages,metadata,id,SBERT CrossEncoder_score,f1_score
99,question: What are some examples of bidirectio...,"BERT, RoBERTa, and SpanBERT",Several examples of bidirectional transformer ...,"[{'id': '3505', 'contributed_by': 'group 2', '...",{},100,0.779548,0.000000
94,question: What are examples of pairwise sequen...,"Recognizing textual entailment, or whether one...","Biological sequence classification, spam detec...","[{'id': '3183', 'contributed_by': 'group 2', '...",{},95,0.208834,0.000000
14,question: Why is the advantage of training the...,Doing so is useful when the task the network i...,"Word2vec embeddings are static embeddings, mea...","[{'id': '2835', 'contributed_by': 'group 4', '...",{},15,0.291513,0.041667
97,question: What cause biases in language models...,"For example, the training data may contain a b...","Several factors, including the number of words...","[{'id': '1720', 'contributed_by': 'group 1', '...",{},98,0.564706,0.058824
10,question: How does backprop (back propagation)...,Backprop computes each of the partial derivati...,Backpropagation through time (BPTT) is a gener...,"[{'id': '2765', 'contributed_by': 'group 10', ...",{},11,0.465275,0.071429
91,question: What is a main difference between ne...,Reasonable classification performance with pre...,Neural classifiers are trained by optimization...,"[{'id': '2855', 'contributed_by': 'group 4', '...",{},92,0.294813,0.076923
93,question: How does pairwise sequence classific...,Append the CLS token to the beginning of the s...,Pairwise sequence classification is the task o...,"[{'id': '3183', 'contributed_by': 'group 2', '...",{},94,0.493120,0.086957
54,question: What is bidirectional RNN answer: <e...,"It combines two independent RNNs, one processi...",A bidirectional RNN for sequence classificatio...,"[{'id': '3209', 'contributed_by': 'group 2', '...",{},55,0.340931,0.086957
37,question: Can features in the CRF depend on ea...,If the features depend on other features than ...,"Feature dependencies are possible, but they ar...","[{'id': '3084', 'contributed_by': 'group 2', '...",{},38,0.445775,0.108108
59,question: What is pre-training answer: <extra_...,It is to first train a transformer language mo...,Pre-training is the process of learning a mode...,"[{'id': '3286', 'contributed_by': 'group 2', '...",{},60,0.584133,0.109091


## WandB




In [ ]:
{"query": "question: What is self-attention answer: <extra_id_0>", "answers": ["The use of self-attention layers allows a network to directly extract and use information from arbitrarily large contexts without the need to pass it through intermediate recurrent connections as in RNNs."], "generation": "Self-attention is a form of attention-based processing that enables the decoder to attend to all inputs within a sequence.

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('wandblearningrate.csv')

In [ ]:
data

,id,step,eval_loss,f1,exact_match
0,bcb66c50-fe97-4790-8ba2-5eb4c490bac9,10,10.066562,2.385153,0
1,bcb66c50-fe97-4790-8ba2-5eb4c490bac9,20,4.669531,9.496017,0
2,bcb66c50-fe97-4790-8ba2-5eb4c490bac9,30,2.583516,17.074071,0
3,bcb66c50-fe97-4790-8ba2-5eb4c490bac9,40,2.166719,22.572582,0
4,bcb66c50-fe97-4790-8ba2-5eb4c490bac9,50,2.142354,26.635682,0
...,...,...,...,...,...
175,ae604fb0-70de-436c-b0d1-f86f73200629,110,3.267021,13.747205,0
176,ae604fb0-70de-436c-b0d1-f86f73200629,120,3.239126,13.026447,0
177,ae604fb0-70de-436c-b0d1-f86f73200629,130,3.235952,12.936986,0
178,ae604fb0-70de-436c-b0d1-f86f73200629,140,3.283860,12.903983,0


In [ ]:
df_last = data.groupby('id').tail(1)


In [ ]:
df_last.shape

(16, 5)

In [ ]:
df_last.to_csv('test.csv')